In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd 
from pathlib import Path
from bioblp.data import COL_EDGE, COL_SOURCE, COL_TARGET

In [10]:
DATA_DIR = Path("../data")
SHARED_DIR = Path("/home/jovyan/workbench-shared-folder/bioblp")

In [7]:
!ls {SHARED_DIR}/models

1baon0eg  iwy9z7m9


## Load benchmark

In [11]:
dpi_benchmark_path = SHARED_DIR.joinpath('data/benchmarks/dpi_fda.tsv') 

In [12]:
dpi_bm = pd.read_csv(dpi_benchmark_path, sep='\t', names=[COL_SOURCE, COL_EDGE, COL_TARGET])
dpi_bm.head(3)

,src,edg,tgt
0,DB01079,DPI,Q13639
1,DB00114,DPI,P20711
2,DB01158,DPI,P13637



* [DB01079; Tegaserod](https://go.drugbank.com/drugs/DB01079)
Tegaserod is a serotonin-4 (5-HT4) receptor agonist indicated for the treatment of constipation predominant irritable bowel syndrome (IBS-C) specifically in women under the age of 65. There is currently no safety or efficacy data for use of tegaserol in men.

* https://www.uniprot.org/uniprotkb/Q13639/entry

In [13]:
dpi_drugs_unique = list(dpi_bm.src.unique())
dpi_prots_unique = list(dpi_bm.tgt.unique())
len(dpi_bm), len(dpi_drugs_unique), len(dpi_prots_unique)

(19161, 2286, 2705)

## Prepare Data for ML

In [14]:
from bioblp.benchmarking.featurize import load_model_and_entity_to_id_maps, ENTITY_TO_ID_MAP, RELATION_TO_ID_MAP

In [26]:
model_dir = Path('/home/jovyan/BioBLP/models/1baon0eg')
model_dir_best = SHARED_DIR.joinpath('models/iwy9z7m9')
kge_artifacts = load_model_and_entity_to_id_maps(model_dir)
entity_to_id_map = kge_artifacts.get(ENTITY_TO_ID_MAP)
relation_to_id_map = kge_artifacts.get(RELATION_TO_ID_MAP)

09:44:18 - INFO - bioblp.benchmarking.featurize - Loading trained model from /home/jovyan/BioBLP/models/1baon0eg


In [27]:
db_keys = {k:v for k,v in entity_to_id_map.items() if k.startswith('DB')}

In [28]:
len(db_keys)

8808

### Generate Negative samples

In [29]:
from bioblp.benchmarking.preprocess import prepare_dpi_samples

In [30]:
pos_neg_df = prepare_dpi_samples(dpi_bm, entity_to_id_map, relation_to_id_map)

In [ ]:
pos_neg_df.head(10)

### Featurize by obtaining joint entity pair encoding

In [29]:
from bioblp.benchmarking.featurize import generate_entity_pair_joint_encoding
from bioblp.benchmarking.featurize import concatenate, average


In [32]:
# pass on to KGEM encoder to featurize each entity pair
generate_entity_pair_joint_encoding(pos_neg_df,
                                    model=kge_artifacts["model"],
                                    transform_fn=concatenate)
                            

NameError: name 'true_pos_df' is not defined